# **Creating results for 2 dimensional Riemann problems**
This notebook is for easily creating reference solutions to use in the thesis. The code is based on code from the development stage, found in `dev_pkg_2dim_refSol.ipynb` which is a notebook used for testing everything while developing.

## **How to produce results**
1. Choose parameters. Be wise on choice of destination to avoid overwriting.
2. Restart kernel and run all cells.

**NB**: `Make sure not to overwrite wanted material, thus choose destination and filename with care`**!!!**

In [ ]:
from datetime import datetime
print("Last run:",datetime.today())

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## **Imports**

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

from riemannsolver import data2d
from riemannsolver import net_mlp2d as network
from riemannsolver import god_mlp2d_newParam as god_aprox
from riemannsolver import netplot
from riemannsolver import function
from riemannsolver import godunov_2d as god_exact_2d

from IPython.display import HTML
from tqdm import tqdm_notebook as tqdm

## **Parameters**

Flux and its derivative in spatial directions.

In [4]:
f = lambda x: x**2/2
dfdu = lambda x: x

g = lambda x: x**2/2
dgdu = lambda x: x

#### Destination and filename 

In [5]:
destination = "res/2dim_newInit/burgers/exact"
name = "circBox_test"

#### Mesh- and method-parameters
To enhance performance, set small $N_y$-value if y-direction in initial data is constant.
* $N_x$ - mesh size in x-direction
* $N_y$ - mesh size in y-direction
* T - temporal maximum
* C - Courant coefficient

In [6]:
Nx = 50
Ny = 50
T = 0.5
C = 0.5

xmin, xmax = -1, 1
ymin, ymax = -1, 1

LeVeque and 'watersplash' initial condition.

In [7]:
init_choice = 1 # 0=LeVeque, 1=watersplash
if init_choice == 0:
    U0 = function.InitialFunc('circ_square').func
    bnd_cond = 'neumann'
elif init_choice == 1:
    U0 = function.InitialFunc('watersplash').func
    bnd_cond = 'periodic'

## **Godunov solver**

In [8]:
god = god_exact_2d.Godunov(
    f=f, dfdU=dfdu, g=g, dgdU=dgdu, U0=U0, 
    x_min=xmin, x_max=xmax, Nx=Nx, 
    y_min=ymin, y_max=ymax, Ny=Ny,
    bnd_cond=bnd_cond, T=T, C=C
)

In [9]:
god.solve

In [10]:
torch.save(god.u,destination+'/u_reference_solution_2D_'+name+'.pt')

## **Animation**

In [11]:
mesh4vid = god.mesh
u4vid  = torch.zeros((len(god.u),god.u[0].size(0),god.u[0].size(1)))
for i in range(len(god.u)):
    u4vid[i] = god.u[i]

In [12]:
flt = netplot.Surface(mesh=mesh4vid, z=u4vid)
HTML(flt.get_anim_color.to_html5_video())